## Installations before the lecture

conda install -c astropy ccdproc (this should also install astropy)<br>
conda install -c anaconda astropy (if you don't have already astropy)<br>
pip install python-magic-bin (if you have a Mac)<br>
pip install dfitspy<br>
pip install mpdaf

# FITS files handling with python

FITS (Flexible Image Transport System) files are a widely used standard format to store images and tables. Python provides tools to create, access and update FITS files. This lecture is largely based on <span style="color:blue">**astropy.io.fits**</span> package, although ways to create and filter list of fits files are provided and a brief overview of other two astropy packages to handle with World Coordinate System (WCS) keywords and with FITS file images is also given.     

## Outline

1. **Accessing FITS files**<br>
[<span style="color:blue">**pathlib**</span>](https://docs.python.org/3/library/pathlib.html) --> Using filesystem paths to create list of files from filenames pattern matching.<br>
[<span style="color:blue">**dfitspy**</span>](https://astrom-tom.github.io/dfitspy/build/html/index.html) --> Quickly search within FITS files headers with a practical command line or within python.<br>
[<span style="color:blue">*ImageFileCollection*</span>](https://ccdproc.readthedocs.io/en/latest/api/ccdproc.ImageFileCollection.html#ccdproc.ImageFileCollection.glob_include) --> Create a collection of fits files to use convenient iterators within the collection.<br>
2. **Working with FITS files headers**<br>
[<span style="color:blue">**astropy.io.fits**</span>](http://docs.astropy.org/en/stable/io/fits/) and (in part) [<span style="color:blue">**astropy.wcs**</span>](http://docs.astropy.org/en/stable/wcs/) packages.
3. **Working with FITS files data**<br>
Again <span style="color:blue">**astropy.io.fits**</span>, but also a bit of [<span style="color:blue">**astropy.nddata**</span>](http://docs.astropy.org/en/stable/nddata/).

## Accessing FITS files

<span style="color:blue">**pathlib**</span> is an object-oriented default python 3 package to manipulate filesystem paths. I do recommend to use it in your codes for the many methods to create, access and rename directories/files.

In [ ]:
from pathlib import Path    # import the main path class

# Let's instantiate a path
p = Path('.')    # relative path
p = Path('/Users/arazza/ESO/Python_Boot_Camp/1904/Day2/')    # absolute path  
print(p.is_absolute())

In [ ]:
p

In [ ]:
# Define a path to the list of FITS files
pathtofits = p.joinpath('fits_files_folder')
print(pathtofits.is_dir())
print(pathtofits.exists())
print(pathtofits)

In [ ]:
# Create a list of paths to each single FITS file at once!
myfits = sorted(pathtofits.glob('*.fits'))
myfits

If you want to display list of files with header keywords in a very efficient way, <span style="color:blue">**dfitspy**</span> is the package you need. It comes with both a command line tool and a normal python importable package. It is important to note that <span style="color:blue">**dfitspy**</span> does not access the FITS files, but it just reads the headers. 

In [ ]:
%%bash
dfitspy -h

In [ ]:
%%bash
dfitspy -f fits_files_folder/*.fits -k OBJECT,FILTER

<span style="color:blue">*ImageFileCollection*</span> is a class of <span style="color:blue">**ccdproc**</span> library.<br>
The latter is an extremely useful library to reduce CCD data by detrending images, computing a bad pixel mask, creating and propagating the errors into a variance map and more! ...<br>
... “But that is another story and shall be told another time.”
― Michael Ende , The Neverending Story

In [ ]:
# Let's import the class and define a collection of FITS files
from ccdproc import ImageFileCollection

# The collection needs the FITS files location and a list of header keywords, as minimum parameters.
images = ImageFileCollection(pathtofits,keywords=['OBJECT','FILTER'])   

In [ ]:
# images.summary is a astropy Table object ... isn't it fantastic?
type(images.summary)

In [ ]:
# Show the table in a notebook ... 
images.summary.show_in_notebook()

In [ ]:
# ... or in a browser
images.summary.show_in_browser()

In [ ]:
# Filter the collection by object type as in the headers
biases = images.files_filtered(object='Bias')
for bias in biases :
    print(bias)

With <span style="color:blue">*ImageFileCollection*</span> you can iterate into the Header Data Unit (HDU), the headers, the <span style="color:blue">*CCDData*</span> objects (see below) or the data of the collection.  

In [ ]:
for data in images.data() :
    print(type(data))

## Exercise

In [ ]:
%pylab tk

Given the convenience function to plot the images

In [ ]:
def plot_data (data) :
    
    '''
    The function accepts an image (numpy.ndarray) as input and return a plot of it with a zscale algorithm
    as in DS9.
    
    Parameters:
    data : numpy.ndarray
    
    Return :
    matplotlib.pyplot.imshow object
    '''
    
    from matplotlib import pyplot as plt
    from mpdaf.tools import zscale
    
    fig = plt.figure()
    vmin, vmax = zscale(data)
    im = plt.imshow(data, origin='lower', cmap='gray_r', vmin=vmin, vmax=vmax)
    
    return im

Plot the images in the **fits_file_folder**

## Working with FITS files headers

We see a practical example where a <span style="color:blue">**BinTableHDU**</span> class ([http://docs.astropy.org/en/stable/io/fits/api/tables.html](http://docs.astropy.org/en/stable/io/fits/api/tables.html)) is instantiated, out of the <span style="color:blue">*images.summary*</span> table created in Section 1.

In [ ]:
from astropy.io import fits

BinHDU = fits.BinTableHDU(images.summary, name='DATA')
print (BinHDU)

In [ ]:
print (repr(BinHDU.header))    # repr() to display the entire header as it appears in the FITS file
# print (repr(BinHDU.data))

Writing out the Bin Table.

In [ ]:
# Create the path to the FITS table file
fitspath = Path('.').joinpath('my_fits_table.fits')
# Save it
fits.writeto(fitspath, BinHDU.data, header=BinHDU.header, overwrite=True)
# ... and let's open the BinHDU file again
hdu = fits.open(fitspath)

Look at the FITS file structure in detail.

In [ ]:
hdu.info()

Reading/Writing the header cards, as the header is a python dictionary!!

In [ ]:
hdr = hdu[1].header
list(hdr.keys())

In [ ]:
hdr['EXTNAME']

In [ ]:
hdr['history'] = 'Test writing on April, 16th 2019'
print (repr(hdr))

The very useful <span style="color:blue">*set*</span> method.

In [ ]:
hdr.set('EXTNAME2', value='MYDATA', comment='Card created during ESOpy 3.0', after='EXTNAME')
print (repr(hdr))

Let's now overwrite the file with the modified header (one proposed way to do it).

In [ ]:
hdu[1].header = hdr
hdu.writeto(fitspath, overwrite=True)

In [ ]:
new_hdr = fits.open(fitspath)[1].header
print(repr(new_hdr))

## Exercise

Access the file **NGC1087_Ha_flux_nosky_mod.fits** in **fits_file_folder** and print the main header (in the extension 0).

We now access the WCS info in the header (you will need for Frederic's lecture!)

In [ ]:
from astropy.wcs import WCS
wcs = WCS(hdr)
print(wcs)

## Working with FITS files data

We now briefly look at how to deal with n-dimensional arrays, e.g. images from FITS files  

In [ ]:
mydata = fits.open(myfile)[0].data # or --> mydata = hdu[0].data

For instance, we can use <span style="color:blue">*CCDData*</span> class. This class allows to define standard deviation map, mask, header and wcs info simply as attributes! ...<br>
... “But that is another story and shall be told another time.”

We only import a FITS file into a <span style="color:blue">*CCDData*</span> class.

In [ ]:
from astropy.nddata import CCDData
import astropy.units as u
ccd = CCDData.read(myfile, unit=u.uJy)

In [ ]:
ccd.unit

In [ ]:
ccd.wcs

In [ ]:
ccd.header

In [ ]:
ccd.uncertainty

**Very Important!** All the image manipulations below can be done equivalently on data from FITS file or on <span style="color:blue">*CCDData*</span> class.

In [ ]:
from astropy.nddata import Cutout2D
from mpdaf.tools import zscale
position = (1061, 1409)
size = (81, 61)
cutout = Cutout2D(ccd, position, size)    # Or equivalently --> cutout = Cutout2D(mydata, position, size)
vmin, vmax = zscale(cutout.data)
plt.imshow(cutout.data, origin='lower', vmin=vmin, vmax=vmax)

In [ ]:
vmin, vmax = zscale(ccd.data)
plt.imshow(ccd, origin='lower', vmin=vmin, vmax=vmax)
cutout.plot_on_original(color='white')

Image resizing.

In [ ]:
from astropy.nddata import block_reduce
smaller = block_reduce(ccd, 16)
vmin, vmax = zscale(smaller)
plt.imshow(smaller, origin='lower', vmin=vmin, vmax=vmax)

There other interesting functions of the package <span style="color:blue">**astropy.nddata**</span>! ...<br>
... “But that is another story and shall be told another time.”

In [ ]:
#myfile = pathtofits.joinpath('NGC1087_Ha_flux_nosky_mod.fits')
#hdu = fits.open(myfile)
#hdr = hdu[0].header